应该是1，从视频提取出csv并保存

In [206]:
# 包含车速信息，代码运行很慢，因为有车速信息，所以必须要逐帧处理

import cv2
import numpy as np
import time
import pandas as pd
import os
import torch

In [207]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.to('cpu')

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [208]:

# 定义检测的类（只检测车辆相关的类别）
classes_to_detect = ['bicycle', 'car', 'motorbike', 'bus', 'truck']


In [209]:
from tools.video import get_video_paths

video_paths = get_video_paths("./dataset/train")

print(video_paths)


['./dataset/train/32.31.250.105/20240501_20240501115227_20240501130415_115227.mp4', './dataset/train/32.31.250.105/20240501_20240501130415_20240501141554_130415.mp4', './dataset/train/32.31.250.105/20240501_20240501141554_20240501152820_141555.mp4', './dataset/train/32.31.250.103/20240501_20240501140806_20240501152004_140807.mp4', './dataset/train/32.31.250.103/20240501_20240501125647_20240501140806_125649.mp4', './dataset/train/32.31.250.108/20240501_20240501113543_20240501135236_113542.mp4', './dataset/train/32.31.250.108/20240501_20240501135236_20240501160912_135235.mp4', './dataset/train/32.31.250.107/20240501_20240501135755_20240501161432_135755.mp4', './dataset/train/32.31.250.107/20240501_20240501114103_20240501135755_114103.mp4']


In [210]:
for path in video_paths:
    file_folder = path.split('/')[-2]
    full_file_folder = str('/').join(path.split('/')[:-1])
    print(file_folder)

32.31.250.105
32.31.250.105
32.31.250.105
32.31.250.103
32.31.250.103
32.31.250.108
32.31.250.108
32.31.250.107
32.31.250.107


In [211]:
def compute_iou(boxA, boxB):
    # boxA 和 boxB 是 [x1, y1, x2, y2] 格式
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # 计算交集面积
    interArea = max(0, xB - xA) * max(0, yB - yA)

    # 计算各自的面积
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # 计算并集面积
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou


In [212]:
# 逐帧处理视频
for path in tqdm(video_paths):
    file_folder = path.split('/')[-2]
    full_file_folder = str('/').join(path.split('/')[:-1])

    # 打开视频文件
    filename = os.path.basename(path)
    video_name = os.path.splitext(filename)[0]
    cap = cv2.VideoCapture(path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # 获取视频帧率

    # 获取视频的帧数
    all_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"视频总帧数: {all_frame_count}")

    # 定义流量线的位置
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    line_position = frame_height // 2  # 根据需要调整位置

    # 初始化变量
    frame_count = 0
    output_data = []
    vehicle_id_counter = 0  # 用于分配新的车辆ID
    vehicles = {}  # 存储车辆信息

    # 定义处理间隔（每25帧处理一次）
    process_frame_interval = 25  # 每25帧处理一次
    output_frame_interval = 25  # 每隔25帧输出一次

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1

        # 仅每隔指定的帧数处理一次
        if frame_count % process_frame_interval != 0:
            continue

        # 使用 YOLOv8 进行推理
        results = model.predict(frame)  # YOLOv8的推理方法
        detections = results[0].boxes  # 获取第一个结果的检测框

        boxes = []
        confidences = []
        class_ids = []

        for box in detections:
            confidence = box.conf.item()  # 获取置信度
            class_id = int(box.cls.item())  # 获取类别ID
            if confidence > 0.3:  # 使用置信度阈值
                class_name = model.names[class_id]  # 获取类别名称
                if class_name in classes_to_detect:
                    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())  # 获取边界框坐标
                    boxes.append([x1, y1, x2, y2])  # 保持原始的坐标形式
                    confidences.append(confidence)
                    class_ids.append(class_id)

        # 车辆跟踪和速度计算
        current_vehicles = {}
        for box in boxes:
            x1, y1, x2, y2 = box
            matched = False
            for vehicle_id, vehicle_data in vehicles.items():
                prev_box = vehicle_data['box']
                iou = compute_iou([x1, y1, x2, y2], prev_box)
                if iou > 0.5:
                    dx = x1 - prev_box[0]
                    dy = y1 - prev_box[1]
                    distance = np.sqrt(dx * dx + dy * dy)
                    # speed = distance * fps  # 像素/秒
                    speed = (distance * 0.01) * fps  # 转换为米/秒

                    current_vehicles[vehicle_id] = {
                        'box': [x1, y1, x2, y2],
                        'speed': speed,
                        'frames': vehicle_data['frames'] + 1
                    }
                    matched = True
                    break
            if not matched:
                vehicle_id_counter += 1
                current_vehicles[vehicle_id_counter] = {
                    'box': [x1, y1, x2, y2],
                    'speed': 0,
                    'frames': 1
                }
        vehicles = current_vehicles

        # 计算密度和流量
        density = len(boxes)
        flow = sum(1 for box in boxes if box[1] <= line_position <= box[1] + box[3])

        speeds = [vehicle['speed'] for vehicle in vehicles.values() if vehicle['speed'] > 0]
        average_speed = sum(speeds) / len(speeds) if speeds else 0

        # 每隔一定帧数输出数据
        if frame_count % output_frame_interval == 0:
            output = {
                'Frame': frame_count,
                'Flow': flow,
                'Density': density,
                'Speed': average_speed
            }
            print(output)
            output_data.append(output)

    cap.release()

    # 保存结果到文件
    df = pd.DataFrame(output_data)
    saved_folder = f"./res/with-speed/{file_folder}"
    if not os.path.exists(saved_folder):
        os.makedirs(saved_folder, exist_ok=True)
    df.to_csv(f'{saved_folder}/{video_name}.csv', index=False)

  0%|          | 0/9 [00:00<?, ?it/s]

视频总帧数: 10770
0: 384x640 4 cars, 2 buss, 1 truck, 1 traffic light, 47.6ms
Speed: 1.8ms preprocess, 47.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
{'Frame': 25, 'Flow': 3, 'Density': 7, 'Speed': 0}
0: 384x640 11 cars, 1 truck, 1 traffic light, 44.2ms
Speed: 1.3ms preprocess, 44.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
{'Frame': 50, 'Flow': 8, 'Density': 12, 'Speed': 0}
0: 384x640 6 cars, 2 trucks, 1 traffic light, 48.8ms
Speed: 1.6ms preprocess, 48.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
{'Frame': 75, 'Flow': 6, 'Density': 8, 'Speed': 3.7102152234373174}
0: 384x640 7 cars, 2 trucks, 45.8ms
Speed: 2.4ms preprocess, 45.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
{'Frame': 100, 'Flow': 8, 'Density': 9, 'Speed': 0.5590169943749475}
0: 384x640 5 cars, 2 trucks, 1 traffic light, 46.6ms
Speed: 1.5ms preprocess, 46.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
{'Frame

 11%|█         | 1/9 [00:37<05:00, 37.60s/it]

{'Frame': 10750, 'Flow': 3, 'Density': 6, 'Speed': 2.9192427107828953}
视频总帧数: 10750
0: 384x640 6 cars, 3 trucks, 52.1ms
Speed: 2.3ms preprocess, 52.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
{'Frame': 25, 'Flow': 5, 'Density': 9, 'Speed': 0}
0: 384x640 7 cars, 1 truck, 50.1ms
Speed: 2.8ms preprocess, 50.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
{'Frame': 50, 'Flow': 4, 'Density': 6, 'Speed': 6.32455532033676}
0: 384x640 10 cars, 52.9ms
Speed: 2.3ms preprocess, 52.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
{'Frame': 75, 'Flow': 6, 'Density': 9, 'Speed': 2.712789905363843}
0: 384x640 5 cars, 1 bus, 1 truck, 52.0ms
Speed: 1.2ms preprocess, 52.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
{'Frame': 100, 'Flow': 3, 'Density': 7, 'Speed': 2.249957131017072}
0: 384x640 2 cars, 1 traffic light, 43.6ms
Speed: 2.4ms preprocess, 43.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 6

 22%|██▏       | 2/9 [01:16<04:29, 38.47s/it]

...

 78%|███████▊  | 7/9 [06:24<02:22, 71.06s/it]

视频总帧数: 27325

0: 384x640 2 cars, 2 trucks, 47.0ms
Speed: 1.2ms preprocess, 47.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
{'Frame': 25, 'Flow': 2, 'Density': 4, 'Speed': 0}
0: 384x640 4 cars, 1 bus, 1 truck, 44.6ms
Speed: 1.9ms preprocess, 44.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
{'Frame': 50, 'Flow': 4, 'Density': 6, 'Speed': 4.028163353192122}
0: 384x640 4 cars, 1 truck, 60.1ms
Speed: 1.5ms preprocess, 60.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
{'Frame': 75, 'Flow': 5, 'Density': 5, 'Speed': 5.9491596045155815}
0: 384x640 (no detections), 42.7ms
Speed: 1.3ms preprocess, 42.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
{'Frame': 100, 'Flow': 0, 'Density': 0, 'Speed': 0}
0: 384x640 3 cars, 43.3ms
Speed: 1.6ms preprocess, 43.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
{'Frame': 125, 'Flow': 2, 'Density': 2, 'Speed': 0}
0: 384x640 (no detections), 41.2ms
Sp

 89%|████████▉ | 8/9 [08:07<01:21, 81.29s/it]

{'Frame': 27325, 'Flow': 9, 'Density': 9, 'Speed': 0}
视频总帧数: 27322
0: 384x640 6 cars, 45.5ms
Speed: 1.3ms preprocess, 45.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
{'Frame': 25, 'Flow': 6, 'Density': 6, 'Speed': 0}
0: 384x640 10 cars, 47.4ms
Speed: 2.9ms preprocess, 47.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
{'Frame': 50, 'Flow': 10, 'Density': 10, 'Speed': 6.551527685790578}
0: 384x640 9 cars, 49.7ms
Speed: 1.1ms preprocess, 49.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
{'Frame': 75, 'Flow': 9, 'Density': 9, 'Speed': 2.6198480522521206}
0: 384x640 12 cars, 44.9ms
Speed: 1.6ms preprocess, 44.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
{'Frame': 100, 'Flow': 12, 'Density': 12, 'Speed': 9.14212160925721}
0: 384x640 8 cars, 177.2ms
Speed: 1.4ms preprocess, 177.2ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)
{'Frame': 125, 'Flow': 8, 'Density': 8, 'Speed': 6.813479

100%|██████████| 9/9 [09:52<00:00, 65.83s/it]
